# 🧠 Job Resume AI Assistant
Welcome to your personal AI hackathon project! This notebook will guide you through building an AI assistant that helps tailor your resume to job descriptions using LangChain and Python.

## 1. 🛠️ Install Required Libraries

In [ ]:
!pip install langchain langgraph openai tiktoken chromadb faiss-cpu pypdf pylatexenc

## 2. 🔑 Set Up API Keys

In [ ]:
import os
from keysNpasswords import openai

# Set your OpenAI key here (or use dotenv or secret manager in production)
os.environ['OPENAI_API_KEY'] = openai

## 3. 📄 Load Your Resume (as Plaintext or Parsed from LaTeX)

In [ ]:
from pathlib import Path

# Load resume from LaTeX template or plaintext
with open("data/resume.txt", "r") as f:
    resume_text = f.read()

print(resume_text[:1000])  # Preview


## 4. 📝 Input a Job Description

In [ ]:
job_description = """Paste a job description here (or load from a file)."""

## 5. 🧠 Compare Resume to Job Using LangChain & Embeddings

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter

# Split and embed
splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = splitter.split_text(resume_text + "\n\n" + job_description)

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_texts(texts, embeddings)

# Search most relevant pieces
query = "What parts of my resume match or mismatch this job description?"
docs = vectorstore.similarity_search(query, k=5)
for i, doc in enumerate(docs):
    print(f"Match {i+1}:
", doc.page_content, "\n")


## 6. ✍️ Rewrite Resume Sections with LLM

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

template = """You're a resume expert. Given a job description and an old resume section, rewrite it to better match the job. Make sure to limit the new resume to 1 page in length.

Job Description:
{job}

Old Resume:
{resume}

Improved Resume:
"""

prompt = PromptTemplate(template=template, input_variables=["job", "resume"])
chain = LLMChain(llm=ChatOpenAI(temperature=0.4), prompt=prompt)

new_summary = chain.run(job=job_description, resume=resume_text)
print(new_summary)


## 7. 📄 Export Updated Resume to LaTeX and PDF

In [ ]:
from string import Template
import subprocess

with open("resume_template.tex", "r") as f:
    latex_template = Template(f.read())

# Replace template fields with new content
filled = latex_template.substitute(
    summary=new_summary,
    experience="TODO: Extract and rewrite experience",
    skills="TODO: Extract and rewrite skills"
)

with open("output/final_resume.tex", "w") as f:
    f.write(filled)

# Compile LaTeX to PDF
subprocess.run(["pdflatex", "-output-directory", "output", "output/final_resume.tex"])
print("✅ Resume PDF generated in output/final_resume.pdf")
